# Bayesian Classifier
In this lab, you will implement and assess the performance of the Bayesian Classifier.

## Lab Instructions:
1. Read the explanation above each requirement very well
2. Read the requirement very well before jumping into the code.
3. Some requirements have essay questions in them, make sure you do NOT miss them.
4. PLEASE Read the hints! They are clear and made to help you complete the requirement as fast as you should 

In [9]:
#### always keep all your imports in the first cell ####
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import csv
import pandas as pd
import math

%matplotlib qt

## Requirement

In this requirement, you will build the Bayesian Classifier and test its performance. 

You are provided with a data file **data1.csv** containing list of points and their corresponding classes. The format of the data files is shown in the table below.

| |Class|Feature 1|Feature 1| 
|-|-|-|-|
|Point#1|1|0.271633|-2.93224|
|Point#2|1|7.020786|-1.98966|
|Point#3|1|2.901827|-0.91291|


You are also provided with a test data file **test_data.csv**. The file contains test points that are unlabelled (i.e. the class to which they belong is unknown).

In [37]:
# TODO [1] : Read the file 'data1.csv' into the variable data.
# data contains the training data together with labelled classes.
def read_data(file_name):
    ## HINT 1: How is the data ordered in the file?
    ## HINT 2: Do you need to cast the data you read from the file?
    data = pd.read_csv(file_name,header=None)
    # The data is random but i will not sort it so that i can use this function to read any data
    # Pandas automatically casted the data to float
    return data
read_data("./data1.csv").sort_values(by=0)


,0,1,2
574,1.0,-9.888939,-13.095984
408,1.0,-5.663155,-5.765111
852,1.0,-5.648197,-6.017975
410,1.0,-7.086853,-7.679691
411,1.0,-7.182050,-12.149323
...,...,...,...
389,3.0,2.276766,0.581083
393,3.0,1.550104,-2.489118
865,3.0,9.180504,1.422481
400,3.0,4.140753,-1.777340


In [40]:
def read_test_data():
    
    # TODO [2.A]: Read the file 'test_data.csv' into the variable test_data
    # test_data contains the unlabelled test class.
    ## HINT: Do you need to cast the data you read from the file?

    test_data = read_data("./test_data.csv")

    # TODO [2.B]: Read the file 'test_data_true.csv' into the variable test_data_true
    # test_data_true contains the actual classes of the test instances, which you will compare
    # against your predicted classes.
    ## HINT: Do you need to cast the data you read from the file?

    test_data_true = read_data("./test_data_true.csv")

    return test_data, test_data_true


read_test_data()


(             0          1
 0    10.701414   3.872536
 1    -3.818318  -5.009778
 2    -3.570719   9.960362
 3     4.943090  -0.015394
 4     4.260826  -0.613494
 ..         ...        ...
 445  12.701387   4.841707
 446   8.134124   8.882378
 447  -2.024167  11.642334
 448   1.069083   1.272935
 449   9.662366  -2.511302
 
 [450 rows x 2 columns],
        0
 0    3.0
 1    1.0
 2    2.0
 3    3.0
 4    3.0
 ..   ...
 445  3.0
 446  3.0
 447  2.0
 448  3.0
 449  3.0
 
 [450 rows x 1 columns])

### Machine Learning Terminlology
Machine learning problems use common termonology (names and notiations) to refer to certain things. It is useful to use this termonology throughout your code to make it readable.

| | |
|:-|:--- |
|$M$:|A scalar; represents the number of training points in the training set.|
|$K$:|A scalar; represents the number of test points in the test set.|
|$N$:|A scalar; represents the number of features of training set/test set (dimensionality of data).|
|$X$:|A numpy array of shape $(M \times N)$ containing the training data **without** its labels, where $M$ is the number of training points and $N$ is the number of features in the dataset (or dimensionality of features). <br/> Each element in $X$ is a tuple $(X_1, X_2, \dots, X_N)$ where $N$ is the number of features in the dataset.| 
|$X_{test}$:| A numpy array of shape $(K \times N)$ containing the test data, where $K$ is the number of test points and $N$ is the number of features in the dataset (or dimensionality of features). <br/> Each element in $X_{test}$ is a tuple $(X_1, X_2, \dots, X_N)$ where $N$ is the number of features in the dataset. <br/> The number of columns in $X_{test}$ is equal to the number of columns in $X$ (because they have the same number of features). However, the number of rows in $X_{test}$ is different to the number of rows in $X$.|
|**$Y$:| A numpy array of shape $(M \times 1)$ containing the labels of the training data. Each row in $Y$ corresponds to the label of the training point in $X$.<br/> For example, $Y[j]$ corresponds to the label of the training point $X[j]$ where $0<=j<M$.|



In [152]:
# TODO [3]: Fill the values of M, K, N, X, XTest, and Y respectively.
# Do not fill them manually (i.e. do not set N = 3). They should be generic for any input file.  
training_data = read_data('data1.csv').sort_values(by=0)
test_data, test_data_true = read_test_data()

training_data = training_data.to_numpy()
test_data = test_data.to_numpy()
test_data_true = test_data_true.to_numpy()

numClasses = 3
M = training_data.shape[0]
N = training_data.shape[1] - 1 # I subtracted 1 because the first column is the class of point and the rest is the features
K = test_data.shape[0]

X = training_data[:,1:]
X_Test = test_data
Y = training_data[:,0]

print("Number of training points : ",M)
print("Number of test points : ",K)
print("Number of features : ",N)
print(f"Shape of training data : ({M},{N})",X.shape,)
print(f"Shape of test data : ({K},{N})",X_Test.shape)
print(f"Shape of labels of training data : ({M},{1})",Y.shape)



[1. 1. 1. ... 3. 3. 3.]
Number of training points :  1150
Number of test points :  450
Number of features :  2
Shape of training data : (1150,2) (1150, 2)
Shape of test data : (450,2) (450, 2)
Shape of labels of training data : (1150,1) (1150,)


In [150]:
def plot(x, y, title='', xlabel='', ylabel='', color_style=[], colors=[], legends='', figure=None, axis=None):
    
    # Add title, x_label, y_label, z_label to axis (~4 lines)
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)

    # Scatter plot of data points with coordinates (x, y, z) with the corresponding color and label (~1 line)
    axis.scatter(x, y,
        c=list(map(lambda c: colors[c-1], color_style.astype(int))))

    # Get the handles and labels for the legend
    handles, labels = axis.get_legend_handles_labels()

    # Add the new legend labels and colors
    handles += [plt.Line2D([], [], marker='o', color=color,linestyle='None') for color in colors]
    labels += legends

    # Create the legend and show the plot
    ax.legend(handles, labels)

    axis.legend(handles, labels)


In [151]:
# TODO [4]: Draw a scatter plot for traning data, where each class is coloured by the colour corresponding 
#           to its index in the colors array.
# Class 1 should be coloured in red, Class 2 should be coloured in green, and Class 3 should be coloured in blue.
# Hint: We have done a similar plot in the previous lab. What operation do we need to select training data 
#       belonging to a certain class?

colors = ['r','g','b']#{0: 'Reds', 1: 'Greens',3:'Blues'} #['r', 'g', 'b', 'c', 'y']

fig = plt.figure()
ax = fig.add_subplot()
plot(X[:, 0], X[:, 1], title='Random Data',
     xlabel='Feature x.', ylabel='Feature y',
     color_style=Y,
     colors=colors,
     legends=["Class 1", "Class 2","Class 3"],
     figure=fig, axis=ax)
# Display the plot
plt.show()


In [ ]:
## What do you notice about the plot? (Write everything you can think of)
'''
    Your Answer:
''' 

### Bayesian Classifier
The Bayesian Classifier calculates the probability of the test point belonging to each class, then the class with highest probability is assigned to the test point.

Classification of $x_{test}$ = $argmax_{i} P\big(C_i|x_{test}\big)$ = $argmax_{i} P(x|C_i) * P(C_i)$

* $P(C_i|x_{test})$: Posterior probability
* $P(x|C_i)$: Class-conditional probability (or distribution)
* $P(C_i)$: Class apriori probability
                
**Note that** $P(C_i|x_{test}) \neq P(x_{test}|C_i) * P(C_i)$. Instead,  $P(C_i|x_{test}) = \frac{P(x_{test}|C_i) * P(C_i)}{P(x_{test})}$. However, when we compare multiple classes, the denominator $P(x_{test})$ is independent of the class $i$ and can be regarded as normalizing factor.

**We start by** computing statistical parameters about each class from the data. 

For each class, we are interested in **three** parameters that will be used for calculating the Gaussian class-conditional distribution and the posterior probability.

These parameters are:

|||
|:-|:-|
|**Class Apriori Probability: ($P_C$)**| A scalar; the probability of class occurence (how frequent this class appears in the training data)|
|**Class Mean: ($\mu$)**| A vector of shape $(N \times 1)$, it is the expected value (mean) calculated from the training points of each class.|
|**Class Covariance Matrix: ($\Sigma$)**| A square symmetric matrix of shape $(N \times N)$ representing the covariances between all the feature calculated from the training points of the class. <br/> For example: Matrix element $\sigma^2_{12}$ is the covariance between the 1st and the 2nd features|



In [190]:
# A list of size (numClasses, 1) containing the a priori probabilities of each class in the training set.
pClasses = [np.count_nonzero(Y == element)/M for element in np.unique(Y)]

rangeIndicesClasses = [[element, np.where(Y == element)[0][0], np.where(Y == element)[
    0][-1]] for element in np.unique(Y)]
print(rangeIndicesClasses)

# A numpy array of size (numClasses, N) containing the mean points of each class in the training set.
estimate_means = [np.mean(X[rangeIndices[1]:rangeIndices[2]],axis=0)for rangeIndices in rangeIndicesClasses]
                    # HINT: USE NP.MEAN
print(estimate_means)

# A numpy array of size (numClasses, N, N) containing the covariance matrices of each class in the training set.
estimate_covariances = [np.cov(X[rangeIndices[1]:rangeIndices[2]])
                        for rangeIndices in rangeIndicesClasses]
                          # HINT: USE NP.COV (Pay attenention for what it takes as an argument)
print(np.array(estimate_covariances).shape)

# for classIndex in range(numClasses):
#     # TODO [5]: Estimate the parameters of the Gaussian distributions of the given classes.
#     # Fill pClasses, estimate_means, and estimate_covariances in this part
#     # Your code should be vectorized WITHOUT USING A SINGLE FOR LOOP.
#     pass

estimate_means = np.array(estimate_means)
estimate_covariances = np.array(estimate_covariances)


[[1.0, 0, 499], [2.0, 500, 899], [3.0, 900, 1149]]
[array([-4.9754611 , -6.02926833]), array([-0.53797489,  9.74339764]), array([ 5.99629515, -0.07639642])]


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.

In [188]:
### Test your implementation ###
### DO NOT CHANGE THIS CODE ###
assert len(pClasses) == numClasses,\
        'Incorrect class apriori probability list, it should be of length {}'.format(len(pClasses))
assert np.sum(pClasses)==1,\
        'Sum of apriori probabilities should be 1, found {}'.format(np.sum(pClasses))

assert estimate_means.shape == (numClasses, N),\
        'Incorrect estimated means, it should be of shape {}'.format((numClasses, N))
assert estimate_covariances.shape == (numClasses, N, N),\
        'Incorrect estimate covariance matrices, it should be of shape {}'.format((numClasses, N, N))

AssertionError: Sum of apriori probabilities should be 1, found 0.9999999999999999

**The second step** in the classifier is to calculate the class-conditional density using the Gaussian destribution:

$P(x|C_i) = \mathcal{N}(x; \mu_i, \Sigma_i) = \frac{1}{(2\pi)^{\frac{N}{2}}|\Sigma_i|^{\frac{1}{2}}} exp\big(\frac{-1}{2}(x-\mu_i)^T\Sigma^{-1}_{i}(x-\mu_i)\big)$

In [ ]:
# TODO 6: Implement the multivariate normal gaussian distribution with parameters mu and sigma, and return the
#  value in prob.
def multivariate_normal_gaussian(X, mu, sigma):
    prob = None
    return prob

In [ ]:
### Test your implementation ###
### DO NOT CHANGE THIS CODE ###
np.random.seed(90)
assertion_x = np.random.rand(3).reshape(-1,1)
assertion_mu = np.random.rand(3).reshape(-1,1)
assertion_sigma = np.random.rand(9).reshape(3,3)
assertion_probability = multivariate_normal_gaussian(assertion_x, assertion_mu, assertion_sigma)[0][0]
assertion_probability = round(assertion_probability, 1)

assert assertion_probability == 7.8,\
    'Incorrect Gaussian Probability calculated'

**The final step** is to go for each test point, calculate its posterior probability against each class, then classify it to the class with the highest posterior probability.

In [ ]:
# TODO [7]: Apply the Bayesian Classifier to predict the classes of the test points.
predicted_classes = [] # predicted_classes: A numpy array of size (K, 1) where K is the number of points in the test set. Every element in this array
                       # contains the predicted class of Bayes classifier for this test point.

for i in range(X_Test.shape[0]):
    print("For test point:", X_Test[i])
    classProbabilities = np.zeros(numClasses)
    # TODO [7.A]: Compute the probability that the test point X_Test[i] belongs to each class in numClasses.
    #  Fill the array classProbabilities accordingly.
    
    
    # TODO [7.B]: Find the prediction of the test point X_Test[i] and append it to the predicted_classes array.
    

    print('Class Probabilities: ', classProbabilities)  # the first class is the left most in the scatter plot
    print("Predicted class is :", predicted_classes[i])
    print("******************************************************************************")


In [ ]:
# TODO [8]: Compute the accuracy of the generated Bayesian classifier 
# WITHOUT USING ANY FOR LOOPs.
accuracy = 0
print('Accuracy = ' + str(round(accuracy,4) * 100) + '%')

In [ ]:
# TODO [9]: Generate a 3D-plot for the generated distributions. x-axis and y-axis represent the features of the data, 
#           where z-axis represent the Gaussian probability N at this point.

x = np.linspace(-10, 10, 300)
y = np.linspace(-10, 15, 300)
X, Y = np.meshgrid(x, y)
Z = np.zeros(X.shape)

for i in range(Z.shape[0]):
    for j in range(Z.shape[1]):
        # TODO [9]: Fill in the matrix Z which will represent the probability distribution of every point.
        # Z[i,j] represents the joint probability N(x,y) for x = i and y = j. 
        # We want to draw the gaussian probability N(x,y) for all points. 
        Z[i, j] = 0

# Make a 3D plot, do not change code
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_surface(X, Y, Z, cmap='viridis', linewidth=0)
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
plt.show()

In [ ]:
# How can you judge your plot is correct?
'''
    Your Answer:
'''